# CAFQA

In [ ]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import circular_ansatz
import numpy as np

In [ ]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
coeffs = np.random.random(len(paulis))

In [ ]:
# define parametrized Clifford circuit that is being optimized over
# here we use the circular_ansatz template
# we fix 2q gates as they will not be optimized over
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)

In [ ]:
# the circuit consists of parametrized gates
for gate in vqe_pcirc.gates:
    print(gate.label, gate.is_fixed())

In [ ]:
# non-fixed gates will be optimized over
# RY and RZ gates can assume 4 values k = 0,1,2,3 which descripe multiples of pi/2

In [ ]:
# the initial parameters are all 0
vqe_pcirc.read()

In [ ]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

In [ ]:
# we can assign a different set of parameters
vqe_pcirc.assign([0,1,2,3,0,1,2,3,0,1,2,3])

In [ ]:
vqe_pcirc.stim_circuit().diagram()

In [ ]:
# we can perform CAFQA by using the main optimization function "claptonize"
ks_best, _, energy_best = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

In [ ]:
# the best parameters are
ks_best

In [ ]:
# with energy
energy_best

In [ ]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.stim_circuit().diagram()

# noisy CAFQA (nCAFQA)

In [ ]:
from clapton.depolarization import GateGeneralDepolarizationModel #TODO: check this out 

In [ ]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.02)

In [ ]:
# after every gate a depol channel is added
vqe_pcirc.stim_circuit().diagram()

In [ ]:
# we can perform nCAFQA by using the main optimization function "claptonize"
# now with the noisy circuit
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

In [ ]:
# the best parameters are
ks_best

In [ ]:
# with noisy/noiseless energy
energy_noisy, energy_noiseless

In [ ]:
# differrence
np.abs(energy_noisy-energy_noiseless)

In [ ]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()